In [1]:
import requests
from splinter import Browser
from bs4 import BeautifulSoup as bs

## Splinter Set Up
*Browser Intentionally does not show.

*Change headless=False to show browser.

### Mac

In [2]:
# https://splinter.readthedocs.io/en/latest/drivers/chrome.html
# !which chromedriver

In [3]:
# executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
# browser = Browser('chrome', **executable_path, headless=True)

### Windows

In [2]:

executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

## Use Splinter to find pages
* Here is the thread I was reading to figure this out:
* https://stackoverflow.com/questions/46468030/how-select-class-div-tag-in-splinter
* It's a pain in the ass to read, but it's right.

In [11]:
url = 'https://www.cars.com/for-sale/searchresults.action/?rd=30&searchSource=QUICK_FORM&stkTypId=28881&zc=77034'
browser.visit(url)



In [12]:
# Find href to click
# https://www.cars.com/vehicledetail/detail/764743599/overview/


divs = browser.find_by_css("div[class='shop-srp-listings__listing-container'] a")

# Save url to use in scraper.
current_url = divs["href"]

print(divs["href"])

https://www.cars.com/vehicledetail/detail/764743599/overview/


In [13]:
# Click link
divs[0].click()

In [44]:
# Advance to next result.

# Apparently if you're going to search by class, all classes must be referenced.
# next_button = browser.find_by_css("a[class='next cui-button--secondary']")

next_button = browser.find_by_css("a[data-linkname='vdp-next']").first
# next_button["href"]
current_url = next_button["href"]
next_button.click()


## Set up Beautiful Soup

In [45]:
url = current_url
response = requests.get(url)
soup = bs(response.text, 'html.parser')
print(response.url)
print(soup.prettify())

https://www.cars.com/vehicledetail/detail/762102222/overview/
<!DOCTYPE doctype html>
<html lang="en">
 <head>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <script>
   if (window.performance && window.performance.clearMarks && window.performance.mark) {
            window.performance.clearMarks();
            window.performance.mark("pageLoadStart");
        }
  </script>
  <link href="https://www.cars.com/vehicledetail/detail/762102222/overview/" rel="canonical"/>
  <link href="https://www.cstatic-images.com" rel="preconnect"/>
  <link href="https://cstatic-graphics.com" rel="preconnect"/>
  <link href="https://fonts.gstatic.com" rel="preconnect"/>
  <link href="https://assets.adobedtm.com" rel="preconnect"/>
  <!--<link rel="preconnect" href="https://cdn.optimizely.com">-->
  <link href="https://api.cars.com" rel="preconnect"/>
  <link href="https://sb.scorecardresearch.com" rel="preconnect"/>
  <link href="https://ad.doubleclick.net" rel="preconnect"/>
  <link href="ht

In [46]:
scraped = {}

basic_info = soup.find_all('li', class_="vdp-details-basics__item")
for res in basic_info:
    try:
        scraped[res.strong.text[:-1]] = res.span.text.lstrip()
        
    except AttributeError as e:
        print(e)

try:
    scraped["price"] = soup.find("div", class_="vehicle-info__price").span.text
    
    header = soup.find_all("h1")[0].text
    head_list = header.split(" ")
    scraped["Year"] = head_list[1]
    scraped["Make"] = head_list[2]
    scraped["Model"] = head_list[3] + " " + head_list[4]
    
    scraped["Seller"] = soup.find("div", class_="page-section--seller-details").h2.text[8:]
    scraped["Seller Address"] = soup.find("div", class_="seller-details-location__text").a["href"]
    scraped["URL"] = response.url

except AttributeError as e:
        print(e)

In [47]:
for entry in scraped:
    print(f"{entry}: {scraped[entry]}")

Fuel Type: Gasoline
Exterior Color: Predawn Gray Mica
City MPG: 22
Interior Color: Black
Highway MPG: 32
Stock: JU503208
Drivetrain: FWD
Transmission: 8-Speed Automatic w/OD
Engine: Regular Unleaded I-4 2.5 L/152
VIN: 4T1B61HK5JU503208
Mileage: 18,370
price: $25,791
Year: 2018
Make: Toyota
Model: Camry XSE
Seller: AutoNation Toyota Gulf Freeway
Seller Address: https://www.google.com/maps/search/?api=1&query=AutoNation%20Toyota%20Gulf%20Freeway%2012111%20Gulf%20Fwy%20Houston%20TX%2077034
URL: https://www.cars.com/vehicledetail/detail/762102222/overview/


In [18]:
database = []

In [48]:
database.append(scraped)

In [49]:
database

[{'Fuel Type': 'Gasoline',
  'Exterior Color': 'Black',
  'City MPG': '17',
  'Interior Color': 'Graphite w/Gun Metal',
  'Highway MPG': '20',
  'Stock': 'GX031530',
  'Drivetrain': '4WD',
  'Transmission': '6-Speed Manual w/OD',
  'Engine': 'Regular Unleaded V-6 3.5 L/211',
  'VIN': '5TFCZ5AN1GX031530',
  'Mileage': '35,876',
  'price': '$31,916',
  'Year': '2016',
  'Make': 'Toyota',
  'Model': 'Tacoma TRD',
  'Seller': 'AutoNation Toyota Gulf Freeway',
  'Seller Address': 'https://www.google.com/maps/search/?api=1&query=AutoNation%20Toyota%20Gulf%20Freeway%2012111%20Gulf%20Fwy%20Houston%20TX%2077034',
  'URL': 'https://www.cars.com/vehicledetail/detail/764743599/overview/'},
 {'Fuel Type': 'Gasoline',
  'Exterior Color': 'Magnetic Gray Metallic',
  'City MPG': '17',
  'Interior Color': 'Black/Graphite',
  'Highway MPG': '22',
  'Stock': 'F5090616',
  'Drivetrain': 'RWD',
  'Transmission': '5-Speed Automatic w/OD',
  'Engine': 'Regular Unleaded V-6 4.0 L/241',
  'VIN': 'JTEZU5JR7F509